In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [2]:
from tensorflow.python.client import device_lib

print(tf.__version__)
print(tf.config.experimental.list_physical_devices('GPU'))
print(device_lib.list_local_devices())

2.7.0
[]
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14893524499487211139
xla_global_id: -1
]


In [3]:
from keras_preprocessing.image import ImageDataGenerator

num_classes = 15

# create a data generator
datagen = ImageDataGenerator()
train_it = datagen.flow_from_directory('images/training', class_mode='categorical', batch_size=64, color_mode='grayscale')
test_it = datagen.flow_from_directory('images/testing', batch_size=1, color_mode='grayscale', shuffle=False)

Found 1500 images belonging to 15 classes.
Found 2985 images belonging to 1 classes.


In [52]:
#define model
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, 5, activation='relu', input_shape=(256,256,1)),
    tf.keras.layers.Conv2D(32, 5, activation='relu'),
    tf.keras.layers.MaxPooling2D(3),
    tf.keras.layers.Conv2D(64, 5, activation='relu'),
    tf.keras.layers.Conv2D(64, 5, activation='relu'),
    tf.keras.layers.MaxPooling2D(3),
    tf.keras.layers.Conv2D(128, 5, activation='relu'),
    tf.keras.layers.Conv2D(128, 5, activation='relu'),
    tf.keras.layers.MaxPooling2D(2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(15)
])
model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [53]:
model.summary()

Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_131 (Conv2D)         (None, 252, 252, 32)      832       
                                                                 
 conv2d_132 (Conv2D)         (None, 248, 248, 32)      25632     
                                                                 
 max_pooling2d_67 (MaxPoolin  (None, 82, 82, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_133 (Conv2D)         (None, 78, 78, 64)        51264     
                                                                 
 conv2d_134 (Conv2D)         (None, 74, 74, 64)        102464    
                                                                 
 max_pooling2d_68 (MaxPoolin  (None, 24, 24, 64)       0         
 g2D)                                                

### Load

In [54]:
model.load_weights("weights3")

### Train

In [43]:
#train model
model.fit(train_it, steps_per_epoch=24, epochs=10) #1500 / 64
model.save_weights('weights')

Epoch 1/10
17/24 [====================>.........] - ETA: 37s - loss: 6.7638 - accuracy: 0.0618

KeyboardInterrupt: 

### Evaluate

In [ ]:
# evaluate model
test_it.reset()
pred_prob = model.predict(test_it, steps=2985)
pred_index = np.argmax(pred_prob,axis=1)

labels = (train_it.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in pred_index]

filenames=test_it.filenames

results=pd.DataFrame({"Filename":filenames,
                      "Predictions":predictions})

results["Filename"] = results["Filename"].str[8:]
results["Filename"] = results["Filename"].str[:-4]
results["Filename"] = pd.to_numeric(results["Filename"]) #.astype(int)
results = results.sort_values(by='Filename', ascending=1)
display(results)

with open('run3.txt', 'w') as f:
    for row in results.iterrows():
        f.write(str(row[1][0])+".jpg "+str(row[1][1])+"\n")


In [42]:
results

,Filename,Predictions
0,0,Office
1,1,Forest
1111,2,Insidecity
2208,3,Forest
2319,4,Coast
...,...,...
2202,2983,kitchen
2203,2984,Suburb
2204,2985,Suburb
2205,2986,Highway
